In [ ]:
%cd /content/drive/MyDrive/final_data_age

/content/drive/MyDrive/final_data_age


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [ ]:
%matplotlib inline
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')
## 기본 세팅 부분

headers = {"X-Naver-Client-Id": "lJrKhZKFXCGXSoYzh7f7",
           "X-Naver-Client-Secret":"YYPQvTkBxR"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"

ages=['20','30','40','50','60']
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df
## 대분류 카테고리 설정
for i in ages:
    categories = [
        {"name": "가구/인테리어", "param": [ "50000004"]},
        {"name": "도서", "param": [ "50005542"]},
        #{"name": "디지털/가전", "param": [ "50000003"]},
        {"name": "생활/건강", "param": [ "50000008"]},
        {"name": "스포츠/레저", "param": [ "50000007"]},
        {"name": "식품", "param": [ "50000006"]},
        {"name": "출산/육아", "param": [ "50000005"]},
        #{"name": "패션의류", "param": [ "50000000"]},
        {"name": "패션잡화", "param": [ "50000001"]},
        {"name": "화장품/미용", "param": [ "50000002"]}
    ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
        {"name": "디지털/가전", "param": [ "50000003"]},
        {"name": "패션의류", "param": [ "50000000"]},
    ],
    "device":"",
    "gender":"",
    "ages":[i]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_a = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_b = change_index(pd.DataFrame(results[1]["data"]), "패션의류")
##
    df_all = pd.merge(df_a, df_b, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all = pd.merge(df_all, df, left_on="period", right_on="period")
## 그래프 그리기
    plt.figure(figsize=[36,10])
    plt.title(i+'대')
    sns.lineplot(data=df_all, markers=True, dashes=False)
## 엑셀에 저장
    if i == '20':
        df_all.to_excel("data_cat_20_all.xlsx", sheet_name="click ratio")
    if i == '30':
        df_all.to_excel("data_cat_30_all.xlsx", sheet_name="click ratio")
    if i == '40':
        df_all.to_excel("data_cat_40_all.xlsx", sheet_name="click ratio")
    if i == '50':
        df_all.to_excel("data_cat_50_all.xlsx", sheet_name="click ratio")
    if i == '60':
        df_all.to_excel("data_cat_60_all.xlsx", sheet_name="click ratio")

Output hidden; open in https://colab.research.google.com to view.